# Convolutional Neural Networks (CNN) - Keras

content is from A-Z Datascience course

## Content:
### Explanation
1. Convolution operation
2. ReLU
3. Max Pooling
4. Flattening
5. Full-connection
Softmax & Cross entropy

### IMPLEMENTATION
1. Data preprocessing
2. Build the Keras model
3. Compile and fit the model
4. Make predictions and determine accuracy

--------- 

Below are key concepts used in Convolutional Neural Networks.

# Convolution Operation
The aim of convolution operation is to reduce the size of an image, by using feature detectors that keep only the specific patterns within the image. Stride is the number of pixels with which we slide the detector. If it is one, we are moving it one pixel each time and recording the value (adding up all the multiplied values). Many feature detectors are used, and the algorithm finds out what is the optimal way to filter images. 3 x 3 feature detector is commonly used, but other sizes can be used.

# ReLU
After feature detectors are applied upon images, ReLU is used to increase non-linearity within images. 

# Max Pooling
Take a 2 x 2 box on the top left corner (starting here), and record the maximum number within the box. Slide it to the right with the stride of 2 (commonly used), and move onto the next row if completed. Repeat this step until all the pixels are evaluated. Aim of max pooling is to keep all the important features even if images have spatial or textual distortions, and also reduce the size which prevents overfitting. So, after applying convolution operation to images, than pooling is applied.

Other pooling techniques are also available such as Mean Pooling, which takes the average of pixels within the box.

# Flattening
Flatten the matrix into a long vector which will be the input to the artificial neural network

# Full Connection
Implement full Artificial Neural Network model to optimize weights.

# Softmax & Cross entropy
Softmax function brings all predicted values to be between 0 and 1, and make them add up to 1. It also comes hand-in-hand with cross-entropy method. 

Just seeing how many wrong predictions the classifier made is not enough to evaluate the performance of ANNs. Instead, Cross Entropy should be used to measure how good the model is, as there can be two models that produce same results while one produced better percentages than the other. For classificaion, Cross Entropy should be used, and for regression, Mean Squared Error should be used. 

---

We will create a dog vs cat classifier. To be able to work with keras library, we need proper structure of images. There should be two folders: Test set and Train set. And, in each folder, cat images and dog images should be placed in two separate folders. In this way, keras will understand how to work with them.


## 0. Read in Data

Data augmentation prevents overfitting, by generating more samples of the images through flipping, rotating, distorting, etc. Keras has built-in Image Augmentation function. To learn more about this function, refer to this [guide](https://keras.io/preprocessing/image/). 

In [8]:
from keras.preprocessing.image import ImageDataGenerator
# https://keras.io/api/preprocessing/image/
    
train_datagen = ImageDataGenerator(rescale = 1./255, #normalizing ==> values now 0~1
                                   shear_range = 0.2, # data augmentation
                                   zoom_range = 0.2, # data augmentation
                                   horizontal_flip = True) # data augmentation


training_set = train_datagen.flow_from_directory('../input/training_set', 
target_size = (64, 64), # Tuple of integers (height, width), 
                        # defaults to (256, 256). The dimensions to which all images found will be resized.
batch_size = 32, # Size of the batches of data (default: 32).
class_mode = 'binary')
# One of "categorical", "binary", "sparse", "input", or None. 
# Default: "categorical". Determines the type of label arrays that are returned: 
#         "categorical" will be 2D one-hot encoded labels, - "binary" will be 1D binary labels, 
#         "sparse" will be 1D integer labels, 
#         "input" will be images identical to input images (mainly used to work with autoencoders).
#         If None, no labels are returned (the generator will only yield batches of image data, 
#                                          which is useful to use with model.predict())

######### ######### ######### ######### ######### ######### ######### ######### 

test_datagen = ImageDataGenerator(rescale = 1./255)

test_set = test_datagen.flow_from_directory('../input/test_set',
                                                target_size = (64, 64),
                                                 batch_size = 32, 
                                                 class_mode = 'binary')

Found 8005 images belonging to 1 classes.
Found 2023 images belonging to 1 classes.


## 1. Build the CNN model

In [9]:
import tensorflow as tf
tf.__version__

'1.13.1'

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D #images are two dimensional. Videos are three dimenstional with time.
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

#initialize the classifier CNN
classifier = Sequential() #Please note that there is another way to build a mode: Functional API.

#applying convolution operation --> build the convolutional layer
classifier.add(Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

#32, 3, 3 --> 32 filters with 3 x 3 for each filter. 
#start with 32 filters, and then create more layers with 64, 128, 256, etc
#expected format of the images.
# 256, 256, 3 --> 3 color channels (RGB), 256 x 256 pixels. But when using CPU, 3, 64, 64 --> due to computational limitation


###################### KERAS Implementation ##############################
# from keras.models import Sequential
# from keras.layers import Convolution2D #images are two dimensional. Videos are three dimenstional with time.
# from keras.layers import MaxPooling2D
# from keras.layers import Flatten
# from keras.layers import Dense

# #initialize the classifier CNN
# classifier = Sequential() #Please note that there is another way to build a mode: Functional API.

# #applying convolution operation --> build the convolutional layer
# classifier.add(Convolution2D(32, 3, 3, input_shape = (64, 64, 3), activation = 'relu'))

# #32, 3, 3 --> 32 filters with 3 x 3 for each filter. 
# #start with 32 filters, and then create more layers with 64, 128, 256, etc
# #expected format of the images.
# # 256, 256, 3 --> 3 color channels (RGB), 256 x 256 pixels. But when using CPU, 3, 64, 64 --> due to computational limitation

In [11]:
#Max Pooling --> create a pooling layer
classifier.add(MaxPool2D(pool_size=2, strides=2)) # Keras: classifier.add(MaxPooling2D(pool_size = (2,2)))
# 2 x 2 size --> commonly used to keep much information.

#add second convolutional layer
classifier.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
classifier.add(MaxPool2D(pool_size=2, strides=2))

#Flattening --> creating a long vector.
classifier.add(Flatten()) #Keras: classifier.add(Flatten()) #no parameters needed.

#classic ANN - full connection
classifier.add(Dense(units=128, activation='relu')) #keras: classifier.add(Dense(output_dim = 128, activation = 'relu'))
#common practice: number of hidden nodes between the number of input nodes and output nodes, and choose powers of 2
classifier.add(Dense(units = 1, activation = 'sigmoid')) #keras: classifier.add(Dense(output_dim = 1, activation = 'sigmoid'))

## 2. Compile the model

In [12]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
# this line of code is same as keras implementation

## 3. Fit the model on images, image preprocessing

In [13]:
classifier.fit(x = training_set, validation_data = test_set, epochs = 25)

### Keras ####
# classifier.fit_generator(training_set, 
#                          samples_per_epoch = 8005, 
#                         nb_epoch = 2, 
#                         validation_data = test_set, 
#                         nb_val_samples = 2025)

Epoch 1/25
251/251 [==============================] - 82s 329ms/step - loss: 0.0026 - acc: 0.9999 - val_loss: 1.0000e-07 - val_acc: 1.0000
Epoch 2/25
251/251 [==============================] - 38s 152ms/step - loss: 1.0000e-07 - acc: 1.0000 - val_loss: 1.0000e-07 - val_acc: 1.0000
Epoch 3/25
251/251 [==============================] - 37s 147ms/step - loss: 1.0000e-07 - acc: 1.0000 - val_loss: 1.0000e-07 - val_acc: 1.0000
Epoch 4/25
251/251 [==============================] - 37s 149ms/step - loss: 1.0000e-07 - acc: 1.0000 - val_loss: 1.0000e-07 - val_acc: 1.0000
Epoch 5/25
251/251 [==============================] - 38s 152ms/step - loss: 1.0000e-07 - acc: 1.0000 - val_loss: 1.0000e-07 - val_acc: 1.0000
Epoch 6/25
251/251 [==============================] - 37s 147ms/step - loss: 1.0000e-07 - acc: 1.0000 - val_loss: 1.0000e-07 - val_acc: 1.0000
Epoch 7/25
251/251 [==============================] - 38s 151ms/step - loss: 1.0000e-07 - acc: 1.0000 - val_loss: 1.0000e-07 - val_acc: 1.0000
Epo

## 4. Making a single prediction

In [22]:
# import numpy as np
# from keras.preprocessing import image
# test_image = image.load_img('../input/cat-and-dog/test_set/cats/cat.4021.jpg',
#                             target_size = (64, 64))

# test_image = image.img_to_array(test_image)
# test_image = np.expand_dims(test_image, axis = 0)

# result = cnn.predict(test_image)

In [23]:
# training_set.class_indices

In [24]:
# if result[0][0] == 1:
#     prediction = 'dog'
# else:
#     prediction = 'cat'

## 4. Improving the model
There are two possible ways of reducing variane, which is making the model fit more to the train set.
* Add more convolutional layers 
    * This will allow more features to be detected prior to fitting to ANN. Make sure to not include input_dim, and include MaxPooling step. Flattening should be at the end of all the layers.
* Add more fully-connected layer (hidden layers)
    * Catches more complex behaviors
    
Thank you for reading this kernel. If you found this helpful, please upvote the kernel or put a short comment below. 